<a href="https://colab.research.google.com/github/rutuja-1462/kaggle-data-pipeline/blob/main/data_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
import sqlite3
from kaggle.api.kaggle_api_extended import KaggleApi
import config

def authenticate_kaggle(username, key):
    """Authenticate with Kaggle using environment variables."""
    os.environ['KAGGLE_USERNAME'] = username
    os.environ['KAGGLE_KEY'] = key
    api = KaggleApi()
    api.authenticate()
    return api

def download_dataset(api, dataset_name, download_path):
    """Download and unzip dataset using Kaggle API."""
    api.dataset_download_files(dataset_name, path=download_path, unzip=True)

def load_dataset(file_path):
    """Load dataset into a Pandas DataFrame."""
    return pd.read_csv(file_path)

def transform_data(data):
    """Transform data into dimension and fact tables."""
    # Product dimension
    product_df = data[['Product line', 'Unit price']].drop_duplicates().reset_index(drop=True)
    product_df['ProductID'] = product_df.index + 1

    # Customer dimension
    customer_df = data[['Gender', 'City']].drop_duplicates().reset_index(drop=True)
    customer_df['CustomerID'] = customer_df.index + 1

    # Fact table
    fact_df = data.merge(product_df, on='Product line').merge(customer_df, on=['Gender', 'City'])
    fact_df = fact_df[['Invoice ID', 'Date', 'ProductID', 'CustomerID', 'Quantity', 'Total']]

    return product_df, customer_df, fact_df

def load_to_sqlite(conn, product_df, customer_df, fact_df):
    """Load data into SQLite database."""
    product_df.to_sql('Product_Dimension', conn, if_exists='replace', index=False)
    customer_df.to_sql('Customer_Dimension', conn, if_exists='replace', index=False)
    fact_df.to_sql('Sales_Fact', conn, if_exists='replace', index=False)

def query_total_sales(conn):
    """Query total sales by product line."""
    query = """
    SELECT
        p."Product line" AS "Product Line",
        ROUND(SUM(f."Total"), 2) AS "Total Sales"
    FROM
        Sales_Fact f
    JOIN
        Product_Dimension p
    ON
        f."ProductID" = p."ProductID"
    GROUP BY
        p."Product line"
    ORDER BY
        "Total Sales" DESC;
    """
    return pd.read_sql_query(query, conn)

def fetch_table_sample(conn, table_name, limit=5):
    """Fetch sample rows from a specific table."""
    query = f"SELECT * FROM {table_name} LIMIT {limit};"
    return pd.read_sql_query(query, conn)

def main():
    """Main function to execute the workflow."""
    # Authenticate with Kaggle
    api = authenticate_kaggle(config.USERNAME, config.PASSWORD)

    # Download and load dataset
    dataset_name = 'aungpyaeap/supermarket-sales'
    download_path = './data'
    download_dataset(api, dataset_name, download_path)

    data = load_dataset(f'{download_path}/supermarket_sales - Sheet1.csv')

    # Transform data
    product_df, customer_df, fact_df = transform_data(data)

    # Load data into SQLite
    with sqlite3.connect('sales_data.db') as conn:
        load_to_sqlite(conn, product_df, customer_df, fact_df)

        # Query example
        sales_summary = query_total_sales(conn)
        print("Total Sales by Product Line:")
        print(sales_summary)

        # Fetch sample data from a table
        sample_data = fetch_table_sample(conn, 'Product_Dimension')
        print("Sample data from Product_Dimension:")
        print(sample_data)

if __name__ == "__main__":
    main()
